In [30]:
import pandas

dat=pandas.read_csv("BankChurners.csv",  delimiter=",")

numerical_list=['Customer_Age','Months_on_book','Credit_Limit',
                'Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt','Total_Trans_Ct',
                'Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio',
                'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']
categorical_list=['Attrition_Flag','Gender','Marital_Status']
ordinal_list=['Dependent_count', 'Education_Level', 'Income_Category', 'Card_Category','Months_Inactive_12_mon','Total_Relationship_Count','Contacts_Count_12_mon']



In [31]:
#Unifying data types

lng=len(dat['Customer_Age'])
have_missing=set()

for el in categorical_list:
    q=list(dict(dat[el].value_counts()).keys())
    for i in range (lng):
        if dat[el][i]=="Unknown":
            dat.at[i,el]=None
            have_missing.add(el)
        else:
            dat.at[i,el]=q.index(dat[el][i])

for el in ['Education_Level','Income_Category','Card_Category']:
    dct={'Education_Level':['Uneducated','High School', 'College','Graduate','Post-Graduate','Doctorate'],
         "Income_Category":['Less than $40K', '$40K - $60K','$60K - $80K', '$80K - $120K', '$120K +'],
         "Card_Category":['Blue', 'Silver', 'Gold', 'Platinum']}
    for i in range (lng):
        if dat[el][i]=="Unknown":
            dat.at[i,el]=None
            have_missing.add(el)
        else:
            dat.at[i,el]=dct[el].index(dat[el][i])

for el in ['Months_Inactive_12_mon','Total_Relationship_Count','Contacts_Count_12_mon']:
    for i in range (lng):
        dat.at[i,el]=int(dat[el][i])           

In [32]:
#data normalization

ln=len(dat["CLIENTNUM"])

for el in numerical_list:
    dat=dat.astype({el:float})
    mn=min(dat[el])
    mx=max(dat[el])
    for i in range(ln):
        dat.at[i,el]=(dat[el][i]-mn)/(mx-mn)

dat2=dat.copy()

In [33]:
#replacing missing values

import numpy
from sklearn.ensemble import RandomForestClassifier
import statistics

dat_no_na=dat.dropna()

for el in have_missing:
    dat_no_na=dat_no_na.astype({el:float})
    rf=RandomForestClassifier()
    idlst=list(range(2,len(dat.columns)))
    for ell in have_missing:
        idlst.remove(list(dat.columns).index(ell))
    tr_x=dat_no_na.iloc[:,idlst]
    tr_y=dat_no_na[el]
    rf.fit(tr_x,tr_y)
    
    data_for_pred=dat.iloc[:,idlst]
    pred=rf.predict(data_for_pred)
    for i in range(len(pred)):
        if dat[el][i]==None:
            dat.at[i,el]=abs(round(pred[i]))

In [35]:
#outliers
cl_dat=dat.copy()

koefs=dict()
for el in numerical_list:
    koefs[el]=3
['Customer_Age','Months_on_book','Credit_Limit',
                'Total_Revolving_Bal','Avg_Open_To_Buy','Total_Amt_Chng_Q4_Q1', 
                 'Total_Trans_Amt','Total_Trans_Ct',
                'Total_Ct_Chng_Q4_Q1','Avg_Utilization_Ratio',
                'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
       'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']
for el in ['Customer_Age','Total_Amt_Chng_Q4_Q1','Total_Ct_Chng_Q4_Q1',
          'Total_Trans_Amt','Total_Trans_Ct']:
    koefs[el]=9

for el in list(numerical_list):
    if el in koefs.keys():
        cl_dat_1=cl_dat[numpy.abs(cl_dat[el]-cl_dat[el].mean())<=(koefs[el]*cl_dat[el].std())]
        cl_dat=cl_dat_1

In [57]:
cl_dat.to_csv("Preprocessed_Upgraded.csv")
dat=pandas.read_csv("Preprocessed.csv")
dat=dat.drop(columns=['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'])
print(dat)

       Unnamed: 0  CLIENTNUM  Attrition_Flag  Customer_Age  Gender  \
0               0  768805383               0      0.404255       1   
1               2  713982108               0      0.531915       1   
2               3  769911858               0      0.297872       0   
3               4  709106358               0      0.297872       1   
4               5  713061558               0      0.382979       1   
...           ...        ...             ...           ...     ...   
10114       10122  772366833               0      0.510638       1   
10115       10123  710638233               1      0.319149       1   
10116       10124  716506083               1      0.382979       0   
10117       10125  717406983               1      0.085106       1   
10118       10126  714337233               1      0.361702       0   

       Dependent_count  Education_Level  Marital_Status  Income_Category  \
0                    3                1               0                2   
1      

In [71]:
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

transformer = GenericUnivariateSelect(mode= "k_best",param=12,score_func=chi2)
sel_data = transformer.fit_transform(dat.drop(columns=["Attrition_Flag"]),list(dat["Attrition_Flag"]))
x_train, x_test, y_train, y_test= train_test_split(sel_data ,list(dat["Attrition_Flag"]))

In [ ]:
from sklearn.metrics import classification_report

rf=RandomForestClassifier(criterion='entropy', max_features='sqrt',
                       min_samples_split=3, n_estimators=1000)
rf.fit(x_train, y_train)
ddd=rf.predict(x_test)
print(classification_report(y_test, ddd,digits=4))

Model accuracy after re-learning with new parameter is a bit higher, but ever this increase is very good result on this ve and learning is faster